In [21]:
'''Restricted Bolzmann machine'''

import numpy as np

class RBM():
    
    def __init__(self,n_visible,n_hidden):
        self.weights = np.random.normal(0,0.1,(n_hidden,n_visible))
        self.bias_v = np.random.normal(0,0.1,(n_visible))  
        self.bias_h = np.random.normal(0,0.1,(n_hidden))
        
    def sample_h(self,X,return_probabilities=False):
        z = X @ self.weights.T + self.bias_h # shape of result: samples x n_hidden
        prob = sigmoid(z)
        if (return_probabilities):
            return prob
        else:
            return bernoulli(prob) 

    def sample_v(self,X,return_probabilities=False):
        z = X @ self.weights + self.bias_v # shape of result: samples x n_visible
        prob = sigmoid(z)
        if (return_probabilities):
            return prob
        else:
            return bernoulli(prob) 
    
    def train(self,x0,xk,ph0,phk,learning_rate=0.01):
        self.weights += learning_rate* (x0.T.dot(ph0) - xk.T.dot(phk)).T
        self.bias_v += learning_rate* (x0 - xk).sum(axis=0)
        self.bias_h += learning_rate* (ph0 - phk).sum(axis=0)
        
def sigmoid(X):
    return 1/(np.exp(-X)+1)    

def bernoulli(X):
    draws = np.random.random(X.shape)
    np.random.random_sample()
    return (X > draws).astype(int)

### Generate dummy recommender data

In [245]:
n_p = 3000
n_m = 40
types = 14

people = np.random.randint(types,size=n_p)
movies = np.random.randint(types,size=n_m)

def get_like(i):
    return bernoulli((1/((movies - people[i])**2+1)))

ratings = np.zeros((len(people),len(movies)))

for i in range(len(ratings)):
    ratings[i] = get_like(i)

# save real values and randomly assign some to -1
X = ratings
X_copy = X.copy()
for i in range(len(ratings)):
    seen = np.random.randint(15,n_m+1)
    not_seen = np.random.choice(np.arange(n_m),n_m-seen,replace=False)
    ratings[i,not_seen] = -1

In [254]:
rbm = RBM(len(movies),20)

epochs = 100
k = 2
batch_size = 100
X = ratings

for epoch in range(epochs):
    ind = 0
    for batch in range(len(X)//batch_size+1):
        x0 = X[ind:ind+batch_size]
        ind+=batch_size
        if (len(x0)==0):
            break
        ph0 = rbm.sample_h(x0,True)
        # xk when k=0
        xk = x0
        for i in range(k):
            # feed into the machine
            hk = rbm.sample_h(xk)
            xk = rbm.sample_v(hk)
            # do not update values that are -1
            xk[x0==-1] = -1
        phk = rbm.sample_h(xk,True)
        rbm.train(x0,xk,ph0,phk)

In [255]:
a = rbm.sample_v(rbm.sample_h(X))

In [256]:
(X_copy == a).mean()

0.79095

In [257]:
(X_copy[X==-1] == a[X==-1]).mean()

0.7810761660248181